In [17]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import h5py
import keras.backend as K
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Activation
from keras import optimizers, layers, callbacks
from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss, coverage_error, average_precision_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelBinarizer
from sklearn.model_selection import GridSearchCV
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix

In [3]:
import pandas as pd
whole = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/ac.csv')
train, test = train_test_split (whole,test_size=0.2)
train.to_csv (path_or_buf = 'C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/train_cat_ac.csv')
test.to_csv (path_or_buf = 'C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/test_cat_ac.csv')

In [ ]:
# Training cost (IM1, IM2, IM3, OS1), (11,12,13,7)
# ivis=46, iev=47, ilc=50, iqt=51, icost=52, icd=54, ipro=55, iothers=57
# CS=58, BT=59, per=60, PT=61, Location=62, yr=63, contract=64

In [4]:
#train set
x_o_train = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/train_cat_ac.csv', 
                    delimiter = ',', usecols = (11,12,13,7),skip_header =2)
y_o_train = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/train_cat_ac.csv', 
                    delimiter = ',', usecols = (46,47,50,51,52,54,55,57),skip_header =2)
CS_train = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/train_cat_ac.csv', 
                    names = ["CS_cat"], header = 1)
BT_train = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/train_cat_ac.csv', 
                    names = ["BT_cat"], header = 1)
per_train = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/train_cat_ac.csv', 
                    names = ["per_cat"], header = 1)
PT_train = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/train_cat_ac.csv', 
                    names = ["PT_cat"], header = 1)
L_train = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/train_cat_ac.csv', 
                    names = ["Location"], header = 1)
yr_train = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/train_cat_ac.csv', 
                    names = ["yr_cat"], header = 1)
contract_train = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/train_cat_ac.csv', 
                    names = ["contract"], header = 1)
#CS_train = CS_train [2:,:]
#BT_train = BT_train [2:,:]
#per_train = per_train [2:,:]
#PT_train = PT_train [2:,:]
#L_train = L_train [2:,:]
#yr_train = yr_train [2:,:]
#contract_train = contract_train [2:,:]

In [5]:
#test set
x_o_test = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/test_cat_ac.csv', 
                    delimiter = ',', usecols = (11,12,13,7),skip_header =1)
y_o_test = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/test_cat_ac.csv', 
                    delimiter = ',', usecols = (46,47,50,51,52,54,55,57),skip_header =1)
CS_test = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/test_cat_ac.csv', 
                    names = ["CS_cat"], header =0)
BT_test = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/test_cat_ac.csv', 
                    names = ["BT_cat"], header =0)
per_test = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/test_cat_ac.csv', 
                    names = ["per_cat"], header =0)
PT_test = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/test_cat_ac.csv', 
                    names = ["PT_cat"], header = 0)
L_test = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/test_cat_ac.csv', 
                    names = ["Location"], header =0)
yr_test = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/test_cat_ac.csv', 
                    names = ["yr_cat"], header =0)
contract_test = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/test_cat_ac.csv', 
                    names = ["contract"], header =0)

In [6]:
lb = LabelBinarizer (neg_label=-1, pos_label=1)
#CS
lb.fit (CS_train)
CS_train = lb.transform (CS_train)
CS_test = lb.transform (CS_test)

#BT
lb.fit (BT_train)
BT_train = lb.transform (BT_train)
BT_test = lb.transform (BT_test)

#per
lb.fit (per_train)
per_train = lb.transform (per_train)
per_test = lb.transform (per_test)


#PT
lb.fit (PT_train)
PT_train = lb.transform (PT_train)
PT_test = lb.transform (PT_test)


#Location
lb.fit (L_train)
L_train = lb.transform (L_train)
L_test = lb.transform (L_test)


#yr
lb.fit (yr_train)
yr_train = lb.transform (yr_train)
yr_test = lb.transform (yr_test)


#contract
lb.fit (contract_train)
contract_train = lb.transform (contract_train)
contract_test = lb.transform (contract_test)

In [7]:
scaler = StandardScaler ()
scaler.fit (x_o_train)
x_o_train = scaler.transform (x_o_train)
x_o_test = scaler.transform (x_o_test)

x_train_split = np.hstack ((x_o_train, CS_train, BT_train, per_train, PT_train, L_train, yr_train, contract_train))
x_test = np.hstack ((x_o_test, CS_test, BT_test, per_test, PT_test, L_test, yr_test, contract_test))

x_train, x_cv, y_train, y_cv = train_test_split (x_train_split,y_o_train,test_size=0.3)

In [36]:
print (x_train.shape)
print (y_train.shape)
print (x_cv.shape)
#print (CS_train.shape)
#print (CS_train)

(226, 11)
(226, 8)
(97, 11)


In [39]:
y_test = y_o_test;
Neuron = 13;
num_classes = 8;
for Neuron in range (10,20):
    model = Sequential ()
    model.add (Dense (units = Neuron, input_dim = 11))
    model.add (layers.Activation ('tanh'))
    model.add (Dense (units = Neuron))
    model.add (layers.Activation ('tanh'))
    model.add (Dense (units = Neuron))
    model.add (layers.Activation ('tanh'))
    model.add (Dense (units = Neuron))
    model.add (layers.Activation ('tanh'))
    model.add (Dense (units = Neuron))
    model.add (layers.Activation ('tanh'))
    model.add (Dense (units = Neuron))
    model.add (Dense (num_classes))
    print (model.summary())

#sgd = optimizer.SGD (lr=0.01, momentum = 0, decay =0, nestrov = False)
    model.compile (loss = 'binary_crossentropy', optimizer = 'adam', metrics=['binary_accuracy'])
    print(model.fit(x_train, y_train, batch_size=80, verbose=1, validation_data=(x_cv, y_cv)))
    print(model.evaluate (x_test, y_test, batch_size = 50, verbose =1))
#help (model.evaluate)
#this is the output units
    y_pred_train = model.predict (x_train)
    y_pred_cv = model.predict (x_cv)
    y_pred_test = model.predict (x_test)
    y_pred_whole = np.vstack ((y_pred_train, y_pred_cv)).astype(np.float32)
    
    print('hamming_loss_train',hamming_loss(y_train, y_train_label))
    print('hamming_loss_cv',hamming_loss(y_cv, y_cv_label))
    print('hamming_loss_test',hamming_loss(y_test, y_test_label))
    print('hamming_loss_traindataset',hamming_loss(y_t, y_t_label))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 10)                120       
_________________________________________________________________
activation_11 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_12 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_13 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 10)                110       
__________

Train on 226 samples, validate on 97 samples
Epoch 1/10
226/226 [==============================] - ETA: 0s - loss: 3.4536 - binary_accuracy: 0.6405 - val_loss: 3.3606 - val_binary_accuracy: 0.6959
Epoch 2/10
226/226 [==============================] - ETA: 0s - loss: 3.2150 - binary_accuracy: 0.6936 - val_loss: 2.9553 - val_binary_accuracy: 0.7010
Epoch 3/10
226/226 [==============================] - ETA: 0s - loss: 2.8293 - binary_accuracy: 0.6952 - val_loss: 2.6641 - val_binary_accuracy: 0.6985
Epoch 4/10
226/226 [==============================] - ETA: 0s - loss: 2.6702 - binary_accuracy: 0.6869 - val_loss: 2.5567 - val_binary_accuracy: 0.6778
Epoch 5/10
226/226 [==============================] - ETA: 0s - loss: 2.4985 - binary_accuracy: 0.6692 - val_loss: 2.3805 - val_binary_accuracy: 0.6508
Epoch 6/10
226/226 [==============================] - ETA: 0s - loss: 2.1857 - binary_accuracy: 0.6554 - val_loss: 1.8109 - val_binary_accuracy: 0.6340
Epoch 7/10
226/226 [=======================

226/226 [==============================] - ETA: 0s - loss: 1.8840 - binary_accuracy: 0.6643 - val_loss: 1.6678 - val_binary_accuracy: 0.6559
Epoch 6/10
226/226 [==============================] - ETA: 0s - loss: 1.7156 - binary_accuracy: 0.6593 - val_loss: 1.6142 - val_binary_accuracy: 0.6508
Epoch 7/10
226/226 [==============================] - ETA: 0s - loss: 1.5893 - binary_accuracy: 0.6521 - val_loss: 1.5015 - val_binary_accuracy: 0.6456
Epoch 8/10
226/226 [==============================] - ETA: 0s - loss: 1.5062 - binary_accuracy: 0.6482 - val_loss: 1.4740 - val_binary_accuracy: 0.6482
Epoch 9/10
226/226 [==============================] - ETA: 0s - loss: 1.4423 - binary_accuracy: 0.6460 - val_loss: 1.4387 - val_binary_accuracy: 0.6495
Epoch 10/10
82/82 [==============================] - ETA: 0s
[1.1731089615240329, 0.67073171894724781]
[ 1.16704079] [-1.05382788] [ 0.03280007] [-5.90706639] [ 1.52116965] [-1.07878117] [-1.4893756] [-2.98675699] [ 0.09791884]
hamming_loss_train 0.39

In [42]:
#save model to json, weights to HDF5
model_json = model.to_json ()
with open ('trainC.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('trainC.h5')
print ('Saved model to disk')

Saved model to disk


In [2]:
#load json and create model
json_file = open('trainC.json', 'r')
loaded_model_json = json_file.read ()
json_file.close ()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('trainC.h5')

In [57]:
y_pred_train = loaded_model.predict (x_train)
y_pred_cv = loaded_model.predict (x_cv)
y_pred_test = loaded_model.predict (x_test)
y_pred_whole = np.vstack ((y_pred_train, y_pred_cv)).astype(np.float32)

In [91]:
from sklearn.metrics import f1_score
parameters = {'k': range (8,9), 's': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]}
score = 'f1_micro'
knn = GridSearchCV (MLkNN(), parameters, scoring =score)
knn.fit (y_pred_train,y_train)
print (knn.best_params_, knn.best_score_)

{'k': 8, 's': 0.4} 0.395343627747


In [92]:
print (knn.predict(y_pred_train))

  (0, 4)	1
  (1, 1)	1
  (1, 4)	1
  (2, 1)	1
  (6, 1)	1
  (6, 4)	1
  (6, 6)	1
  (7, 4)	1
  (7, 6)	1
  (8, 4)	1
  (9, 4)	1
  (12, 1)	1
  (12, 6)	1
  (16, 1)	1
  (20, 1)	1
  (23, 1)	1
  (24, 1)	1
  (24, 6)	1
  (25, 1)	1
  (26, 1)	1
  (27, 4)	1
  (28, 1)	1
  (28, 2)	1
  (29, 6)	1
  (30, 1)	1
  (30, 6)	1
  (31, 6)	1
  (32, 1)	1
  (33, 1)	1
  (34, 4)	1
  (34, 6)	1
  (38, 6)	1
  (39, 1)	1
  (39, 2)	1
  (39, 6)	1
  (40, 1)	1
  (40, 2)	1
  (41, 2)	1
  (42, 1)	1
  (43, 4)	1
  (45, 1)	1
  (45, 4)	1
  (45, 5)	1
  (45, 6)	1
  (46, 1)	1
  (48, 6)	1
  (49, 1)	1
  (51, 1)	1
  (51, 2)	1
  (52, 1)	1
  (53, 1)	1
  (54, 1)	1
  (54, 4)	1
  (56, 1)	1
  (58, 1)	1
  (60, 1)	1
  (60, 2)	1
  (61, 1)	1
  (61, 4)	1
  (61, 6)	1
  (62, 1)	1
  (62, 6)	1
  (63, 4)	1
  (63, 6)	1
  (66, 1)	1
  (68, 1)	1
  (71, 4)	1
  (71, 6)	1
  (72, 1)	1
  (74, 1)	1
  (75, 4)	1
  (76, 1)	1
  (76, 4)	1
  (77, 1)	1
  (77, 5)	1
  (78, 1)	1
  (78, 2)	1
  (80, 1)	1
  (83, 4)	1
  (83, 6)	1
  (84, 1)	1
  (85, 4)	1
  (87, 1)	1
  (87, 4)	1
  (

In [93]:
print (knn.predict_proba(y_pred_train))

  (0, 0)	0.000755857898715
  (0, 1)	0.0961228787316
  (0, 2)	0.0903623452124
  (0, 3)	0.037249123747
  (0, 4)	0.0145236204366
  (0, 5)	0.0588320955293
  (0, 6)	0.0522302030682
  (0, 7)	0.000195963158926
  (1, 0)	0.000755857898715
  (1, 1)	0.117578878448
  (1, 2)	0.0439144107574
  (1, 3)	0.0412117964861
  (1, 4)	0.0914133756889
  (1, 5)	0.0302331602026
  (1, 6)	0.0522302030682
  (1, 7)	0.000195963158926
  (2, 0)	0.000755857898715
  (2, 1)	0.0703756790713
  (2, 2)	0.0439144107574
  (2, 3)	0.033286451008
  (2, 4)	0.0316102327148
  (2, 5)	0.0588320955293
  (2, 6)	0.0185332978629
  (2, 7)	0.000195963158926
  (3, 0)	0.000755857898715
  (3, 1)	0.0188812797508
  (3, 2)	0.0903623452124
  (3, 3)	0.037249123747
  (3, 4)	0.099956681828
  (3, 5)	0.0506609711503
  (3, 6)	0.0606544293695
  (3, 7)	0.000195963158926
  (4, 0)	0.000755857898715
  (4, 1)	0.0188812797508
  (4, 2)	0.0903623452124
  (4, 3)	0.00158506909562
  (4, 4)	0.0316102327148
  (4, 5)	0.0588320955293
  (4, 6)	0.0606544293695
  (4, 7)	0.